# Recommendation System Example Code
First we need to load the ratings...

In [151]:
import pandas as pd
import numpy as np
from math import sqrt
from breakinghits_script import pull_dataframe, ratings, show_tables
import pickle

In [7]:
rat = ratings()

In [43]:
rat.head(5)

,user_id,user_music_id,date_added,rating
0,298,15,2017-09-14 13:19:12,4
1,298,16,2017-09-14 13:19:25,4
2,298,7,2017-09-14 16:25:09,5
3,298,17,2017-09-15 16:25:11,4
4,296,7,2017-09-15 16:30:55,5


###### Sci-kit Learn Surprise Example
Now we can train and cross-validate models to get the best model. Chose 3 fold data split.

In [152]:
from surprise import SVD, KNNBasic
from surprise import Dataset, Reader
from surprise import evaluate, print_perf, prediction_algorithms
import pandas as pd

In [5]:
# A reader is still needed but only the rating_scale param is requiered.
reader = Reader(rating_scale=(1, 5))
# The columns must correspond to user id, item id and ratings (in that order).
data = Dataset.load_from_df(ratings[['user_id', 'user_music_id', 'rating']], reader)
# build the trainset
trainset = data.build_full_trainset()
# choose the number of folds - 3 chosen.
#data.split(3)
testset = trainset.build_testset() # create the test set

In [21]:
from surprise import KNNBasic, KNNWithMeans, KNNBaseline
from surprise import SVD
from surprise import SlopeOne
from surprise import CoClustering
from surprise.accuracy import rmse, fcp

In [7]:
knnb = KNNBasic(sim_options={'user_based': False})
knnw = KNNWithMeans(sim_options={'user_based': False})
knnbas = KNNBaseline(sim_options={'user_based': False})
svd =SVD()
so =SlopeOne()
cc = CoClustering()

In [8]:
knnb.train(trainset)
knnw.train(trainset)
knnbas.train(trainset)
svd.train(trainset)
so.train(trainset)
cc.train(trainset)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.


###### Calculate the RMSE and FCP
NOTE: The lower the RMSE the better. The higher the FCP the better

In [9]:
print('----- KNN Basic -----'); rmse(knnb.test(testset));fcp(knnb.test(testset));
print('----- KNN Means -----'); rmse(knnw.test(testset));fcp(knnw.test(testset));
print('----- KNN Baseline -----'); rmse(knnbas.test(testset));fcp(knnbas.test(testset));
print('----- SVD -----'); rmse(svd.test(testset));fcp(svd.test(testset));
print('----- SVD++ -----'); rmse(so.test(testset));fcp(so.test(testset));
print('----- Co-Clustering -----'); rmse(cc.test(testset));fcp(cc.test(testset));

----- KNN Basic -----
RMSE: 0.5253
FCP:  0.9331
----- KNN Means -----
RMSE: 0.5143
FCP:  0.7777
----- KNN Baseline -----
RMSE: 0.4880
FCP:  0.8384
----- SVD -----
RMSE: 0.7062
FCP:  0.8311
----- SVD++ -----
RMSE: 0.5083
FCP:  0.8774
----- Co-Clustering -----
RMSE: 0.5962
FCP:  0.6796


Appears that the KNN works pretty well... However, over time KNN is going to be computationally expensive. KNN Baseline or SVD++ seem to be strongest candidates...

### Recommendation Predictions
Now we can choose a model and create predictions for a given user.

In [153]:
# Load the prewritten functions for recommendation system
from breakinghits_script import bh_model, bh_predict, bh_top_recs, bh_song_names

In [11]:
# first let's establish the model using pred_rec
svd = bh_model(ratings)

RMSE: 0.9186


In [12]:
# create a list of prediction tuples
predictions = bh_predict(svd, 319, ratings)

In [13]:
# get the top 5 recommendations from the recommendations
num_pred = bh_top_recs(predictions)
num_pred

,Song_ID,rating
0,18,4.478044
1,7,4.363012
2,48,4.349521
3,9,4.294630
4,53,4.254476


In [14]:
# converts the id's to the names of the songs...
bh_song_names(num_pred)

,title,rating
0,LOVE DEALER,4.363012
1,Wishing You The Worst,4.294630
2,WHEN I WAS ME,4.478044
3,LOVE DEALER [MUSIC VIDEO],4.349521
4,Love Dealer [TwoDB mix],4.254476
